In [1]:
import warnings
warnings.filterwarnings('ignore')

# Day 18 - It starts with a tensor

## The tensor API

* Most operations can also be called as exactly equivalent tensor methods

In [2]:
import torch

a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

* This is equivalent to the following:

In [3]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

* The [online docs](http://pytorch.org/docs) are exhaustive, and group the operations
* Creation (`zeros`, `ones`, ...)
* Indexing, slicing mutating (`transpose`, ...)
* Math
    * Pointwise
    * Reduction
    * Comparison
    * Spectral (frequencies)
    * Other special functions
    * BLAS and LAPACK, standardized linear algebra operations
* Random sampling
* Serializtion
* Parallelism (setting parameters, eg. `set_num_threads`)

## Tensors: Scenic views of storage

* The values are stored in a `torch.Storage`, which holds a 1D, contiguous chunk of memory
* Actual `torch.Tensor`s are views over this storage
* Each tensor can have different offsets and per-dimension strides

### Indexing into storage

In [4]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage() # This is depracated, and untyped_storage should be used instead

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [5]:
points_storage = points.storage()
points_storage[0]

4.0

In [6]:
points.storage()[1]

1.0

* Of course, changing the underlying storage will also change what a tensor views

In [7]:
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

In [8]:
bools = torch.tensor([[True, False], [False, False], [False, True], [True, True]])
bools.untyped_storage() # <- Just one byte!!

 1
 0
 0
 0
 0
 1
 1
 1
[torch.storage.UntypedStorage(device=cpu) of size 8]

### Modifying stored values: In-place operations

* Some operations are only available as methods on `Tensor` objects
* These are identified by their trailing undescores, like `zero_()`
* They are *in-place* operations, modifying the underlying data
* Thus, they do not create and return a new tensor

In [9]:
a = torch.ones(3, 2)
a

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [10]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## Tensor metadata: Size, offset, and stride

* A tensor is fully defined by storage, size, offset and stride
* Size is a tuple, indicating the number of elements along each dimension
* Offset is the index of the tensor's first element in the underlying storage
* Stride is the number of elements that need to be skipped to get to the next element in each dimension

### Views of another tensor's storage

* To access the second point, the offset has to be 2, as the first two elements of the storage have to be skipped

In [11]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [12]:
second_point.size(), second_point.shape

(torch.Size([2]), torch.Size([2]))

* Two elements have to be skipped to get to the next point
* One element has to be skipped to get to the next coordinate of a point

In [13]:
points.stride()

(2, 1)

* Accessing the element at `i, j`, we retrieve `storage_offset + i * stride[0] + j * stride[1]`
* Defining a tensor like this makes many operations cheap, by simply allocating a new tensor with different size, offset, and stride, viewing the same underlying memory

In [14]:
second_point = points[1]
second_point.size(), second_point.storage_offset(), second_point.stride()

(torch.Size([2]), 2, (1,))

* Sometimes, we want to modify the tensor, without changing the original data
* In order to do this, we can clone the tensor

In [15]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

### Transposing without copying

* The `t` function is a shorthand for `transpose`-ing a two-dimensional tensor
* This allows us, for example, to turn our points array from one where each row represents a point, to one where each column represents a point

In [16]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [17]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

* These share the same untyped storage, but differ in their properties

In [18]:
id(points.untyped_storage()) == id(points_t.untyped_storage())

True

In [19]:
points.stride(), points_t.stride()

((2, 1), (1, 2))

* Transposing is simply flipping the shape and stride of the tensor

### Transposing in higher dimensions

* To transpose in higher dimensions, we just have to specify the dimensions along which to flip the shape and stride

In [20]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape, transpose_t.shape

(torch.Size([3, 4, 5]), torch.Size([5, 4, 3]))

In [21]:
some_t.stride(), transpose_t.stride()

((20, 5, 1), (1, 5, 20))

* A tensor that is laid out in memory starting from the rightmost dimension onward can be efficiently visited element by element, as it is `contiguous`
* This locality can provide the best performance

### Contiguous tensors

* Some operations work only on contiguous tensors
* Trying to use these will inform us to use `contiguous`, which costs nothing if the tensor already is contiguous

In [22]:
points.is_contiguous(), points_t.is_contiguous()

(True, False)

* To obtain a contiguous tensor from a non-contiguous one, the `contiguous` method will change the stride, as well as the underlying storage to match it

In [23]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t, points_t.stride(), points_t.storage()

(tensor([[4., 5., 2.],
         [1., 3., 1.]]),
 (1, 2),
  4.0
  1.0
  5.0
  3.0
  2.0
  1.0
 [torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6])

In [24]:
points_t_cont = points_t.contiguous()
points_t_cont, points_t_cont.stride(), points_t_cont.storage()

(tensor([[4., 5., 2.],
         [1., 3., 1.]]),
 (3, 1),
  4.0
  5.0
  2.0
  1.0
  3.0
  1.0
 [torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6])

* Essentially, transposing changes the view, while making a transpose `contiguous` changes the underlying storage to match the original stride

## Moving tensors to the GPU

* PyTorch provides hardware acceleration not just with CUDA, but also ROCm, or Google TPUs and Intel's XPUs

### Managing a tensor's device attribute

* In addition to `dtype`, a tensor also has a `device` attribute

In [25]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device="cuda")

* Aside from specifying it during creation, a tensor can be moved `to` a different device

In [26]:
points_gpu = points.to(device="cuda")

* With multiple GPUs, they can be directly indexed

In [27]:
points_gpu = points.to(device="cuda:0")

* Moving it back to the CPU works the same way

In [28]:
points_cpu = points_gpu.to("cpu")

* There are shorthand methods for this

In [29]:
points_gpu = points.cuda()
points_gpu = points.cuda(0)
points_cpu = points_gpu.cpu()

## NumPy interoperability

* Conversion between NumPy's `ndarray`s and PyTorch's `tensor`s is zero-copy

In [30]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

* As long as the data is stored in CPU RAM, this operation is effectively no cost, as the NumPy array shares the same underlying memory
* There is of course also an operation that does the reverse
* Note that NumPy's default type is `float64`

In [31]:
import numpy as np

points_np_fresh = np.ones((3, 4))

points = torch.from_numpy(points_np)
points_fresh = torch.from_numpy(points_np_fresh)

points.dtype, points_fresh.dtype

(torch.float32, torch.float64)

## Generalized tensors are tensors, too

* Anything that fulfills the `tensor` API can be considered a tensor
* There are specialized tensor types, for example for different hardware
* One type is the sparse tensor, which stores only nonzero elements and index information
* The $dispatching$ mechanism finds the correct operation to perform on whatever the underlying data is
* Later, we meet $quantized$ tensors, which are different from the $dense$, or $strided$ tensors that we already know
* The number of kinds of tensors has been growing steadily

## Serializing tensors

* To store tensors in a file, PyTorch uses `pickle` under the hood

In [32]:
torch.save(points, "./DLPT/data/ourpoints.t")

In [33]:
with open("./DLPT/data/ourpoints.t", "wb") as f:
    torch.save(points, f)

In [34]:
points = torch.load("./DLPT/data/ourpoints.t")

In [35]:
with open("./DLPT/data/ourpoints.t", "rb") as f:
    points = torch.load(f)

* This file format is not interoperable, and can generally only be read by PyTorch

### Serializing to HDF5 with h5py

* When interoperability is needed, tensors can be stored in the HDF5 format
* This is achieved with the `h5py` library, after converting to a NumPy array

In [36]:
import h5py

f = h5py.File("./DLPT/data/ourpoints.hdf5", "w")
dset = f.create_dataset("coords", data=points.numpy())
f.close()

* Here, `"coords"` is the key into the HDF5 file
* We can use other keys as well, and we can index into the file while it's on disk

In [37]:
f = h5py.File("./DLPT/data/ourpoints.hdf5", "r")
dset = f["coords"]
last_points = dset[-2:]

* The data is only read from disk once we index into it by asking for `[-2:]`
* The returned object is a NumPy-like array with the same API, allowing us to use `torch.from_numpy`
* This then copies the data into a PyTorch `Storage`

In [38]:
last_points = torch.from_numpy(dset[-2:])
f.close()

## Conclusion

* We have learned everything we need to represent all of our data in floats
* other aspects, like creating views, indexing tensors with other tensors, and broadcasting, are covered later as need arises
* In the next chapter, we will represent real-world data with tensors
    * We start off with tabular data
    * But we will also move on to something more elaborate

## Exercises

1. Create a tensor `a` from `list(range(9))`. Predict and then check the size, offset, and stride.
    1. Create a new tensor using `b = a.view(3, 3)`. What does view do? Check that `a` and `b` share the same storage.
    2. Create a tensor `c = b[1:, 1:]`. Predict and then check the size, offset, and stride.

In [39]:
a = torch.tensor(list(range(9)))

The size should be `[9]`, the offset `0`, and the stride `[1]`. (Nope! This is a tuple.)

In [40]:
a.size(), a.storage_offset(), a.stride()

(torch.Size([9]), 0, (1,))

In [41]:
b = a.view(3, 3)

My guess is that it might create a tensor of size `[3, 3]`, which would look like this:
```py
[[0, 1, 2],
 [3, 4, 5],
 [6, 7, 8]]
```

In [42]:
b, a.untyped_storage() == b.untyped_storage()

(tensor([[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]]),
 True)

I was correct.

In [43]:
c = b[1:, 1:]

The size should be `[2, 2]`, the offset `3`$^1$, and the stride `(3, 1)`. If I'm correct, the tensor should look like this:
```py
[[4, 5],
 [7, 8]]
```
$^1$Ah, off-by-one. My beloved!

In [44]:
c, c.size(), c.storage_offset(), c.stride()

(tensor([[4, 5],
         [7, 8]]),
 torch.Size([2, 2]),
 4,
 (3, 1))

2. Pick a mathematical operation like cosine or square root. Can you find a corresponding function in the `torch` library?
    1. Apply the function element-wise to `a`. Why does it return an error?
    1. What operation is required to make the function work?
    1. Is there a version of your function that operates in place?

I'll pick atan. [Here](https://pytorch.org/docs/stable/generated/torch.atan.html) we go!

In [45]:
try:
    a.atan_()
except Exception as e:
    print(e)

result type Float can't be cast to the desired output type Long


The error is telling us exactly what's needed, so let us cast the type with `float`.

In [46]:
a = a.float()
a.atan_()

tensor([0.0000, 0.7854, 1.1071, 1.2490, 1.3258, 1.3734, 1.4056, 1.4289, 1.4464])

I used the in-place version above, to produce the error. I'm assuming that, when the exercise was written, functions like `torch.atan` or `a.atan` would have caused an error, but now, only the in-place version does. Also, yes. There is a version of this function that operates in place. See above.